# Exercise 02: Homework Assignment 1.3

## Task

Write a program that calculates how often product pairs occur together in the same shopping basket.  With `J` products, this produces a matrix of size `JxJ`.  Use the Instacart market basket data to test your program.  Use the variables order_id and product_id.

In [1]:
import tqdm

import scipy.sparse

import numpy as np
import pandas as pd

## Test data

Create some data and refrence result.  The reference solution is created in a very naïve way taht is bullet proof.  Not efficient but sufficient for small data.  We can compare proper solutions with this ground truth.

In [2]:
np.random.seed(501)

In [3]:
n_baskets = 8
n_products = 8
max_n_products = 4

In [4]:
basket_list = []
for _ in range(n_baskets):
    size_b = np.random.choice(range(2, max_n_products))
    basket_list.append(np.random.choice(n_products, size_b, replace=False))
basket_list

[array([4, 2, 6]),
 array([6, 4, 3]),
 array([0, 7, 4]),
 array([7, 2]),
 array([3, 2, 7]),
 array([0, 4]),
 array([6, 1]),
 array([4, 7])]

In [5]:
basket_df = pd.DataFrame({
    "basket": np.repeat(range(len(basket_list)), [len(x) for x in basket_list]),
    "product": [y for x in basket_list for y in x],
})
basket_df.head()

,basket,product
0,0,4
1,0,2
2,0,6
3,1,6
4,1,4


In [6]:
truth = np.zeros((n_products, n_products), dtype=int)

for b in basket_list:
    for j in b:
        for i in b:
            truth[i, j] += 1

# diagonal is the count of baskets that products occur in
assert np.all(
    basket_df.sort_values("product").groupby("product").basket.count().values
    == np.diag(truth)[np.diag(truth)>0]
)

# matrix is symmetric
assert np.all(truth == truth.T)

truth

array([[2, 0, 0, 0, 2, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 3, 1, 1, 0, 1, 2],
       [0, 0, 1, 2, 1, 0, 1, 1],
       [2, 0, 1, 1, 5, 0, 2, 2],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 2, 0, 3, 0],
       [1, 0, 2, 1, 2, 0, 0, 4]])

## Computation of co-occurrences using list

In [7]:
# turn the input data frame into a list of baskets, each basket is a list of products
# slightly more than we need now, but we'll see this function again in one of the future exercises
def baskets_df_to_list(
    x,
    variable_basket="basket",
    variable_product="product",
    min_basket_size=0,
    shuffle=False,
    to_string=False,
    seed=501,
):

    # create raw basket list
    x_basket_product = x[[variable_basket, variable_product]]
    n_products = x[variable_product].max() + 1
    keys, values = x_basket_product.sort_values(variable_basket).values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index)
    basket_list = [list(set(a)) for a in arrays[1:]]

    # format basket list
    basket_list_out = []
    for basket in basket_list:
        if len(basket) >= min_basket_size:
            if to_string:
                basket_list_out.append([str(x) for x in basket])
            else:
                basket_list_out.append(basket)

    # randomise basket order and product order in baskets
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(basket_list_out)
        for i in range(len(basket_list_out)):
            np.random.shuffle(basket_list_out[i])

    return basket_list_out, n_products

# for each basket increase the co-occurrence counts for the products in the basket by 1
def co_occurrences_list(x, variable_basket="basket", variable_product="product"):
    basket_list, n_products = baskets_df_to_list(
        x=x,
        variable_basket=variable_basket,
        variable_product=variable_product,
        min_basket_size=0,
        shuffle=False,
        to_string=False,
    )

    co_occurrences = np.zeros((n_products, n_products), dtype=int)
    for b in tqdm.tqdm(basket_list):
        co_occurrences[np.ix_(b, b)] += 1

    return co_occurrences

In [8]:
co_occurrences = co_occurrences_list(basket_df)
assert np.all(truth == co_occurrences)
co_occurrences

100%|██████████| 8/8 [00:00<00:00, 7699.50it/s]


array([[2, 0, 0, 0, 2, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 3, 1, 1, 0, 1, 2],
       [0, 0, 1, 2, 1, 0, 1, 1],
       [2, 0, 1, 1, 5, 0, 2, 2],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 2, 0, 3, 0],
       [1, 0, 2, 1, 2, 0, 0, 4]])

## Computation of co-occurrences using sparse matrix

In [9]:
# create a (binary) space matrix that indicates whether a basket (row) contains a product (col)
# co-occurrences are the dot produdct of this basket-product matrix
def co_occurrences_sparse(x, variable_basket="basket", variable_product="product"):
    row = x[variable_basket].values
    col = x[variable_product].values
    dim = (x[variable_basket].max()+1, x[variable_product].max()+1)

    basket_product_table = scipy.sparse.csr_matrix(
        (np.ones(len(row), dtype=int), (row, col)),
        shape=dim
    )
    co_occurrences_sparse = basket_product_table.T.dot(basket_product_table)
    co_occurrences_dense = co_occurrences_sparse.toarray()
    return co_occurrences_dense

In [10]:
co_occurrences_2 = co_occurrences_sparse(basket_df)
assert np.all(truth == co_occurrences_2)
co_occurrences_2

array([[2, 0, 0, 0, 2, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 3, 1, 1, 0, 1, 2],
       [0, 0, 1, 2, 1, 0, 1, 1],
       [2, 0, 1, 1, 5, 0, 2, 2],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 2, 0, 3, 0],
       [1, 0, 2, 1, 2, 0, 0, 4]])

## Apply to a real world data set

In [11]:
PATH = "/private/data/teaching/instacart_exercise_2"

In [12]:
products = pd.read_parquet(f"{PATH}/products.parquet")
n_products = products.shape[0]

baskets_df_instacart = pd.read_parquet(f"{PATH}/baskets.parquet")

In [13]:
baskets_df_instacart.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
3,2,45918,4,1
7,2,1819,8,1
4,2,30035,5,0


In [14]:
f"{baskets_df_instacart.order_id.nunique():,}"

'3,214,874'

In [15]:
co_occurrences = co_occurrences_list(
    x=baskets_df_instacart,
    variable_basket="order_id",
    variable_product="product_id",
)

100%|██████████| 3214874/3214874 [04:13<00:00, 12666.23it/s]


In [16]:
co_occurrences_2 = co_occurrences_sparse(
    x=baskets_df_instacart,
    variable_basket="order_id",
    variable_product="product_id",
)

In [17]:
# only use a (random) subset of product pairs so the comparison is quicker
idx = np.random.choice(co_occurrences_2.shape[0], 5_000)
assert np.all(
    co_occurrences[np.ix_(idx, idx)]
    == co_occurrences_2[np.ix_(idx, idx)]
)

## Some time measurements

In [18]:
%load_ext line_profiler

In [19]:
baskets_df_instacart_sample = baskets_df_instacart[baskets_df_instacart["order_id"]<10_000]

In [20]:
%lprun -f co_occurrences_list co_occurrences_list(x=baskets_df_instacart_sample, variable_basket="order_id", variable_product="product_id")

100%|██████████| 9428/9428 [00:02<00:00, 4457.60it/s]


Timer unit: 1e-06 s

Total time: 8.39968 s
File: <ipython-input-7-267f15c8aca5>
Function: co_occurrences_list at line 40

Line #      Hits         Time  Per Hit   % Time  Line Contents
    40                                           def co_occurrences_list(x, variable_basket="basket", variable_product="product"):
    41         2    6289788.0 3144894.0     74.9      basket_list, n_products = baskets_df_to_list(
    42         1          2.0      2.0      0.0          x=x,
    43         1          0.0      0.0      0.0          variable_basket=variable_basket,
    44         1          1.0      1.0      0.0          variable_product=variable_product,
    45         1          1.0      1.0      0.0          min_basket_size=0,
    46         1          1.0      1.0      0.0          shuffle=False,
    47         1          1.0      1.0      0.0          to_string=False,
    48                                               )
    49                                           
    50       

In [21]:
%lprun -f co_occurrences_sparse co_occurrences_sparse(x=baskets_df_instacart_sample, variable_basket="order_id", variable_product="product_id")

Timer unit: 1e-06 s

Total time: 0.985196 s
File: <ipython-input-9-4a29580ae69f>
Function: co_occurrences_sparse at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def co_occurrences_sparse(x, variable_basket="basket", variable_product="product"):
     4         1        208.0    208.0      0.0      row = x[variable_basket].values
     5         1         19.0     19.0      0.0      col = x[variable_product].values
     6         1       1126.0   1126.0      0.1      dim = (x[variable_basket].max()+1, x[variable_product].max()+1)
     7                                           
     8         2       8340.0   4170.0      0.8      basket_product_table = scipy.sparse.csr_matrix(
     9         1        401.0    401.0      0.0          (np.ones(len(row), dtype=int), (row, col)),
    10         1          2.0      2.0      0.0          shape=dim
    11                                               )
    12         1  

## Pure sparse implementation

So actually if we keep the data sparse we can speed up the computation significantly.

#### Implement sparse calculation, return pandas data frame

In [22]:
# create a (binary) space matrix that indicates whether a basket (row) contains a product (col)
# co-occurrences are the dot produdct of this basket-product matrix
# return data frame
def co_occurrences_sparse_2(x, variable_basket="basket", variable_product="product"):
    row = x[variable_basket].values
    col = x[variable_product].values
    dim = (x[variable_basket].max()+1, x[variable_product].max()+1)

    basket_product_table = scipy.sparse.csr_matrix(
        (np.ones(len(row), dtype=int), (row, col)),
        shape=dim
    )
    co_occurrences_sparse = basket_product_table.T.dot(basket_product_table).tocoo()
    co_occurrences_df = pd.DataFrame({
        "product_1": co_occurrences_sparse.row,
        "product_2": co_occurrences_sparse.col,
        "co-occurrence": co_occurrences_sparse.data,
    })
    return co_occurrences_df

In [23]:
%lprun -f co_occurrences_sparse_2 co_occurrences_sparse_2(x=baskets_df_instacart_sample, variable_basket="order_id", variable_product="product_id")

Timer unit: 1e-06 s

Total time: 0.060717 s
File: <ipython-input-22-89ec51a841c8>
Function: co_occurrences_sparse_2 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def co_occurrences_sparse_2(x, variable_basket="basket", variable_product="product"):
     5         1         39.0     39.0      0.1      row = x[variable_basket].values
     6         1         11.0     11.0      0.0      col = x[variable_product].values
     7         1        889.0    889.0      1.5      dim = (x[variable_basket].max()+1, x[variable_product].max()+1)
     8                                           
     9         2       5074.0   2537.0      8.4      basket_product_table = scipy.sparse.csr_matrix(
    10         1        146.0    146.0      0.2          (np.ones(len(row), dtype=int), (row, col)),
    11         1          1.0      1.0      0.0          shape=dim
    12                                               )
    13       

#### produce result

In [24]:
co_occurrences_df = co_occurrences_sparse_2(
    x=baskets_df_instacart_sample,
    variable_basket="order_id",
    variable_product="product_id"
)
co_occurrences_df = co_occurrences_df.sort_values(["product_1", "product_2"]).reset_index()
co_occurrences_df

,index,product_1,product_2,co-occurrence
0,65,1,1,6
1,3599,1,196,1
2,14225,1,769,1
3,68266,1,3798,1
4,77418,1,4210,1
...,...,...,...,...
1114398,677850,49685,30489,1
1114399,774587,49685,34939,1
1114400,1114393,49685,49685,1
1114401,252370,49686,12204,1


#### Check whether the results is correct

In [25]:
reference_result = co_occurrences_list(
    x=baskets_df_instacart_sample,
    variable_basket="order_id",
    variable_product="product_id"
)

100%|██████████| 9428/9428 [00:01<00:00, 6595.68it/s]


In [26]:
matrix_from_co_occurrences_df = np.zeros(reference_result.shape)
matrix_from_co_occurrences_df[(co_occurrences_df["product_1"], co_occurrences_df["product_2"])] = co_occurrences_df["co-occurrence"].values
assert np.all(reference_result == matrix_from_co_occurrences_df)

&mdash; <br>
Dr. Sebastian Gabel <br>
Machine Learning in Marketing &ndash; Exercise 2 <br>
2020 <br>